In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf 
#tf.test.gpu_device_name()


In [9]:
trainset = pd.read_csv('../input/trainSetH1N1.csv', index_col="id")
trainset_s = pd.read_csv('../input/trainSetSeason.csv', index_col="id")
testset = pd.read_csv('../input/testSet.csv', index_col="id")

In [11]:
X = trainset.loc[:, trainset.columns != 'target'].values
X_test1 = testset.iloc[:, :].values
y_h = trainset.loc[:, 'target'].values
y_s = trainset_s.loc[:, 'target'].values


### Training Season dataset

In [12]:
ann_s = tf.keras.models.Sequential()
ann_s.add(tf.keras.layers.Dense(64, activation='selu'))
ann_s.add(tf.keras.layers.Dense(units=32, activation='selu'))
ann_s.add(tf.keras.layers.Dense(units=16, activation='selu'))
ann_s.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
ann_s.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
ann_s.fit(X, y_s, batch_size = 1024, epochs = 1000)

ep - loss: 0.4519 - accuracy: 0.7910
Epoch 808/1000
27/27 [==============================] - 0s 952us/step - loss: 0.4553 - accuracy: 0.7878
Epoch 809/1000
27/27 [==============================] - 0s 947us/step - loss: 0.4556 - accuracy: 0.7910
Epoch 810/1000
27/27 [==============================] - 0s 966us/step - loss: 0.4605 - accuracy: 0.7878
Epoch 811/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.4548 - accuracy: 0.7893
Epoch 812/1000
27/27 [==============================] - 0s 951us/step - loss: 0.4530 - accuracy: 0.7916
Epoch 813/1000
27/27 [==============================] - 0s 947us/step - loss: 0.4551 - accuracy: 0.7892
Epoch 814/1000
27/27 [==============================] - 0s 982us/step - loss: 0.4522 - accuracy: 0.7913
Epoch 815/1000
27/27 [==============================] - 0s 974us/step - loss: 0.4520 - accuracy: 0.7919
Epoch 816/1000
27/27 [==============================] - 0s 972us/step - loss: 0.4509 - accuracy: 0.7920
Epoch 817/1000
27/27 [=======

In [15]:
y_prob = ann_s.predict(X)
y_prob_s = ann_s.predict(X_test1)

from sklearn.metrics import roc_curve, roc_auc_score
roc_auc_score(y_s, y_prob)

0.8724069843680411


### Training H1N1 dataset

In [16]:
ann_h = tf.keras.models.Sequential()
ann_h.add(tf.keras.layers.Dense(64, activation='selu'))
ann_h.add(tf.keras.layers.Dense(units=32, activation='selu'))
ann_h.add(tf.keras.layers.Dense(units=16, activation='selu'))
ann_h.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [17]:
ann_h.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
ann_h.fit(X, y_h, batch_size = 1024, epochs = 1000)

=========] - 0s 1ms/step - loss: 0.3660 - accuracy: 0.8433
Epoch 806/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3674 - accuracy: 0.8424
Epoch 807/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3663 - accuracy: 0.8432
Epoch 808/1000
27/27 [==============================] - 0s 978us/step - loss: 0.3662 - accuracy: 0.8435
Epoch 809/1000
27/27 [==============================] - 0s 986us/step - loss: 0.3659 - accuracy: 0.8441
Epoch 810/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3666 - accuracy: 0.8442
Epoch 811/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3674 - accuracy: 0.8429
Epoch 812/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3665 - accuracy: 0.8433
Epoch 813/1000
27/27 [==============================] - 0s 1ms/step - loss: 0.3679 - accuracy: 0.8427
Epoch 814/1000
27/27 [==============================] - 0s 954us/step - loss: 0.3660 - accuracy: 0.8430
Epoch 815/1000
27

In [19]:
y_prob = ann_h.predict(X)
y_prob_h = ann_h.predict(X_test1)
from sklearn.metrics import roc_curve, roc_auc_score
roc_auc_score(y_h, y_prob)

0.8525690808547142

In [20]:
submission_dataset = pd.read_csv('../input/submission_format.csv', index_col="respondent_id")
submission_dataset.head()


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.5,0.7
26708,0.5,0.7
26709,0.5,0.7
26710,0.5,0.7
26711,0.5,0.7


In [23]:
np.testing.assert_array_equal(testset.index.values, submission_dataset.index.values)
submission_dataset["h1n1_vaccine"] = y_prob_h
submission_dataset["seasonal_vaccine"] = y_prob_s
submission_dataset.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.090218,0.186823
26708,0.080526,0.043085
26709,0.660689,0.829925
26710,0.597209,0.848034
26711,0.232076,0.421505


In [24]:
submission_dataset.to_csv('../model_preds/submissionNN.csv', index=True)
